Esse código será dividido 6 em trechos, que serão responsáveis, por ordem, em importar e criar o caminho de comunicação para o modelo, criar a classe agente, que conterá esse caminho, criar as ferramentas que serão usadas, criar o prompt para o modelo ententer os passos do ReAct, a lógica do loop para completar o ReAct, e, por fim, um exemplo de execução do agente.

In [109]:
%pip install groq

import os
from groq import Groq 

client = Groq(api_key="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")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Nesse trecho foi instalado o pacoto do groq, importado o pacote de variáveis de ambiente e definida a variável que vai contar a chave API importada do groqcloud.

In [110]:
class Agent:
    def __init__(self, client, system_prompt):
        self.client = client
        self.system_prompt = system_prompt
        self.messages = []
        if self.system_prompt is not None:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def __call__(self, prompt):
        if prompt is not None:
            self.messages.append({"role": "user", "content": prompt})
        resultado = self.execute()
        self.messages.append({"role": "assistant", "content": resultado})
        return resultado
    
    def execute(self):
        completion = client.chat.completions.create(messages=self.messages, model="llama-3.3-70b-versatile")
        return completion.choices[0].message.content

Seguindo o mesmo código mostrado no video, pois são bem simples e, também, as mensagens enviadas para o modelo devem seguir uma estrutura pré-determinada, seguindo a mesma explicação dada no código do vídeo, o trecho de código cria a classe "Agent". O primeiro método é um construtor (__init__) que passa os parâmetros cliente e system para os atributos da classe instanciada, e, se for passado um parâmetro para system, quer dizer que uma informação do papel "system" vai ser passada para o modelo, colocando na lista de messages. O segundo método é um __call__, ou seja, vai ser chamada toda vez que o objeto for chamado também, se tiver uma mensagem, ela vai ser colocada na lista tendo "usuário" como papel, e também usa a função execute, colocando o resultado dessa função no final da lista de mensagens. E na última função, execute, é solicitado um completion ao modelo usado, semelhantemente ao que foi usado anteriomente.

In [111]:
def calcular(operacao):
    return eval(operacao)

def checar_preco_dos_produtos(produto):
    match produto.lower():
        case "maçã":
            return "10 reais por quilo"
        case "banana":
            return "6 reais por quilo"
        case "mamão":
            return "5 reais por quilo"
        case "uva":
            return "14 reais por quilo"
        case "melancia":
            return "4 reais por quilo"
        case "batata":
            return "7 reais por quilo"
        case "tomate":
            return "9 reais por quilo"
        case "morango":
            return "15 reais por quilo"
        case "cenoura":
            return "7 reais por quilo"
        case "repolho":
            return "3 reais por quilo"

Nesse código são construídas as ferramentas que o modelo vai ter acesso. Esse caso de uso é um exemplo mais real de uma aplicação do modelo em estado ReAct, onde é apresentado uma lista de produtos, com o nome e o preço desses e a possibilidade de se fazer cálculos com eles.

In [112]:
system_prompt = """
Você vai funcionar em um ciclo de Pensamento, Ação, PAUSA, Observação.
No final do ciclo você imprime um Resposta.
Use Pensamento para descrever seus pensamentos sobre a questão que lhe foi perguntada.
Use ação para mostrar uma das ações disponíveis para você (ferramentas) - depois retorne PAUSA
A Observação será demonstrada no início de cada output a partir do segundo, e ele vai ser o resultado de uma das seguintes ferramentas.

Suas ações disponíveis serão:

calcular:
exemplo dado: calcular: 3 * 5 / 9
Roda um cálculo e retorna o resultado

checar_preco_dos_produtos:
exemplo dado: checar_preco_dos_produtos: banana
retorna o preço de um quilo da banana

Sessão de exemplo:

Pergunta: Quanto eu vou ter que pagar se eu comprar 2 quilos de banana e 3 quilos de mamão?

Pensamento: Primeiro preciso achar o preço de um quilo de banana
Ação: checar_preco_dos_produtos: Banana
PAUSA

Você será chamado novamente com isso:

Observação: 6 reais por quilo
Pensamento: eu preciso multiplicar esse valor por 2 (quantidade de banana)
Ação: calcular: 6 * 2
PAUSE

Você será chamado novamente com isso:

Observação: Dois quilos de banana são 12 reais
Pensamento: Agora que sei o preço de dois quilos de bananas, preciso saber o preço de um quilo de mamão
Ação: checar_preco_dos_produtos: mamão
PAUSE

Observação: O preço de um quilo de mamão é 5 reais
Pensamento: Agora que sei o preço de um quilo de mamão, preciso multiplicar esse valor pela quantidade de mamão (3 quilos)
Ação: calcular: 5 * 3
PAUSE

Observação: 3 quilos de mamão são 15 reais
Pensamento: Agora que sei o preço de dois quilos de bananas e 3 quilos de mamão, preciso somar esses dois valores
Ação: calcular: 15 + 12
PAUSE

Se você tiver a resposta, imprima como a Resposta.

Resposta: O valor a ser pago será de 27 reais.

Agora é a sua vez:
""".strip()

Nesse trecho é apresentado o prompt que será usado, que foi baseado no apresentado no video, porém traduzido, feito para o agente funcionar em estado de ReAct.

In [ ]:
import re

def loop_agente(iteracoes_maximas, prompt_de_sistema, pergunta_inicial):
    agente = Agent(client, prompt_de_sistema)
    i = 0
    proximo = pergunta_inicial
    ferramentas = ['calcular', 'checar_preco_dos_produtos']
    while i < iteracoes_maximas:
        i+=1
        resultado = agente(proximo)
        print(resultado)
        if "Resposta" in resultado:
            return
        if "PAUSA" in resultado and "Ação" in resultado:
            acao = re.findall(r"Ação: ([a-z_]+): (.+)", resultado, re.IGNORECASE)
            if acao[0][0] in ferramentas:
                resultado_ferramenta = eval(f"{acao[0][0]}('{acao[0][1]}')")
                proximo = f"Observação: {resultado_ferramenta}"
            else:
                proximo = "Ferramenta não encontrada"
        if i == iteracoes_maximas - 1:
            print("Número máximo de iterações atingidas")

Realiza a lógica de loop do ReAct usando um máximo de iterações possíveis. Cada iteração é enviado um prompt a mais para a lista de mensagens (função __call__ do objeto) e é feito uma série de "checagens" para esse resultado, identificando se nessa sentenã tem a Resposta, qual ferramenta é usada, a aplicação da operação para essa ferramenta, como exemplo.

In [ ]:
loop_agente(10, system_prompt, "Qual valor eu vou ter que pagar se eu comprar 2 quilos de banana, 10 quilos de morango e 7 quilos de batata?")

Pensamento: Primeiro, preciso achar o preço de um quilo de banana, pois isso será necessário para calcular o custo total da compra. 
Ação: checar_preco_dos_produtos: banana
PAUSA
Pensamento: Eu preciso multiplicar o preço de um quilo de banana por 2, pois estou comprando 2 quilos de banana.
Ação: calcular: 6 * 2
PAUSA
Pensamento: Agora que sei o preço de 2 quilos de banana, preciso achar o preço de um quilo de morango para calcular o custo total de 10 quilos de morango.
Ação: checar_preco_dos_produtos: morango
PAUSA
Pensamento: Eu preciso multiplicar o preço de um quilo de morango por 10, pois estou comprando 10 quilos de morango.
Ação: calcular: 15 * 10
PAUSA
Pensamento: Agora que sei o preço de 2 quilos de banana e 10 quilos de morango, preciso achar o preço de um quilo de batata para calcular o custo total de 7 quilos de batata.
Ação: checar_preco_dos_produtos: batata
PAUSA
Pensamento: Eu preciso multiplicar o preço de um quilo de batata por 7, pois estou comprando 7 quilos de batat

Finalmente, a única linha de código (chamada do loop) incluí o máximo de iterações, o prompt que usamos para configurar o modelo para agir em ReAct e a pergunta sobre o preço das compras. Além disso, é possível ver toda a linha de raciocínio feita pela IA.